In [1]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 58 (delta 25), reused 45 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 10.84 MiB | 14.56 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [2]:
!cd {REPO_NAME} && git pull

Already up to date.


In [3]:
import os
os.chdir('ASR/Script')

In [4]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:34<00:00, 580.39it/s]



train split summary:
  Total images: 20103



Saving val: 100%|██████████| 2522/2522 [00:04<00:00, 586.72it/s]



val split summary:
  Total images: 2511



Saving test: 100%|██████████| 2522/2522 [00:04<00:00, 585.13it/s]


test split summary:
  Total images: 2512

  Images after filter : 25126



In [5]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 43.7/235.7 GB disk)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ultralytics import YOLO
print("GPU available:", torch.cuda.is_available())

GPU available: True


# Active YOLO functions

In [7]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [8]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [9]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [10]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



# Hyperparameters

In [11]:
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20103
N_INITIAL = int(0.3 * Num_Train)
N_PER_PHASE = int(0.1 * Num_Train)
NUM_PHASES = 5

# MC-Dropout

In [12]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import numpy as np
from tqdm import tqdm

transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    tensor = transform(image)
    return tensor.cuda() if torch.cuda.is_available() else tensor


def mc_dropout_inference(model, images, num_passes=2, device='cuda'):
    """
    Perform MC Dropout inference and return mean + variance across predictions.
    """
    model.train()
    for module in model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.eval()

    images = images.to(device)
    predictions = []

    with torch.no_grad():
        for _ in range(num_passes):
            pred = model(images)  # [B, num_classes]
            predictions.append(pred)

    preds_tensor = torch.stack(predictions)  # [T, B, num_classes]
    mean_preds = preds_tensor.mean(dim=0)    # [B, num_classes]
    var_preds = preds_tensor.var(dim=0).mean(dim=1)  # [B] average variance per sample

    return mean_preds, var_preds

In [13]:
def select_uncertain_samples(unlabeled_images, model, num_passes=2, k=10, batch_size=32, original_dir="/content/data_yolo/train", device='cuda'):
    """
    Batched version of MC Dropout uncertainty sampling.
    Args:
        unlabeled_images: list of (class_name, filename)
        model: PyTorch model
        Returns:
            selected_images, uncertainty_scores, selected_indices
    """
    model.to(device)

    paths = [os.path.join(original_dir, cls, fn) for cls, fn in unlabeled_images]
    uncertainty_scores = []
    all_indices = []

    for i in tqdm(range(0, len(paths), batch_size), desc="Evaluating batches"):
        batch_paths = paths[i:i+batch_size]
        batch_samples = unlabeled_images[i:i+batch_size]

        batch_tensors = []
        valid_indices = []

        for j, path in enumerate(batch_paths):
            if not os.path.exists(path):
                print(f"[Warning] Missing file: {path}")
                continue
            try:
                tensor = preprocess_image(path)
                batch_tensors.append(tensor)
                valid_indices.append(i + j)
            except Exception as e:
                print(f"[Error] Failed to load {path}: {e}")

        if not batch_tensors:
            continue

        batch_tensor = torch.stack(batch_tensors)  # [B, C, H, W]
        _, batch_uncertainty = mc_dropout_inference(model, batch_tensor, num_passes=num_passes, device=device)

        uncertainty_scores.extend(batch_uncertainty.cpu().tolist())
        all_indices.extend(valid_indices)

    # Final selection
    uncertainty_scores = np.array(uncertainty_scores)
    all_indices = np.array(all_indices)
    top_indices_within_valid = np.argsort(uncertainty_scores)[-k:]
    selected_indices = all_indices[top_indices_within_valid]
    selected_images = [unlabeled_images[i] for i in selected_indices]

    return selected_images, uncertainty_scores, selected_indices.tolist()

In [14]:
# from tqdm import tqdm

# def select_uncertain_samples(unlabeled_images, model, num_passes=5, k=10, original_dir="/content/data_yolo/train"):
#     """
#     Select k most uncertain images from unlabeled_images using MC Dropout.
#     """
#     uncertainty_scores = []

#     for class_name, filename in tqdm(unlabeled_images, desc="Evaluating uncertainty"):
#         image_path = os.path.join(original_dir, class_name, filename)
#         image_tensor = preprocess_image(image_path)
#         _, uncertainty = mc_dropout_inference(model, image_tensor, num_passes=num_passes)
#         score = uncertainty.mean().item()
#         uncertainty_scores.append(score)

#     uncertainty_scores = np.array(uncertainty_scores)
#     selected_indices = np.argsort(uncertainty_scores)[-k:]  # top-k uncertain
#     selected_images = [unlabeled_images[i] for i in selected_indices]

#     return selected_images, uncertainty_scores, selected_indices


In [15]:
import csv

# Initialize dataset
setup_active_dataset()
all_samples = get_all_samples()

# Prepare CSV logging
log_path = "accuracy_log.csv"
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Phase", "Num_Images", "Top-1 Accuracy", "Top-5 Accuracy"])

# Initial sampling and train
current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

print("Initial Epoch")
model = YOLO('yolo11n-cls.pt')
results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name="active-phase0", project="runs/classify")

# Log initial accuracy
acc1 = results.top1
acc5 = results.top5

with open(log_path, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([0, len(current_sample_list), acc1, acc5])

# Compute remaining samples
remaining_samples = list(set(all_samples) - set(current_sample_list))

# Active Learning (Uncertainty Sampling)
for phase in range(1, NUM_PHASES):
    n_to_add = min(N_PER_PHASE, len(remaining_samples))
    if n_to_add <= 0:
        print("No more samples")
        break

    print(f"Phase {phase}: selecting top {n_to_add} uncertain samples...")

    # Load model from previous phase
    model = YOLO(f"runs/classify/active-phase{phase-1}/weights/last.pt")

    # Apply uncertainty sampling using MC Dropout
    # Send to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.model = model.model.to(device)
    model.model.train()  # Enable dropout
    for module in model.model.modules():
        if isinstance(module, nn.BatchNorm2d):
            module.eval()

    selected_images, uncertainty_scores, selected_indices = select_uncertain_samples(
        remaining_samples,
        model.model,
        num_passes=2,
        k=n_to_add,
        batch_size=32,
        device=device
    )

    new_samples = [remaining_samples[i] for i in selected_indices]
    copy_samples(new_samples)

    current_sample_list.extend(new_samples)
    remaining_samples = list(set(remaining_samples) - set(new_samples))

    print(f"Phase {phase}: add {n_to_add} samples, total sample number: {len(current_sample_list)}")

    # Train model for this phase
    results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name=f"active-phase{phase}", project="runs/classify")

    # Log accuracy
    acc1 = results.top1
    acc5 = results.top5

    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([phase, len(current_sample_list), acc1, acc5])

    print(f"Phase {phase+1} training completed")


Complete creating train_dir /content/data_active/train
Initial Epoch


100%|██████████| 5.52M/5.52M [00:00<00:00, 98.3MB/s]

Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_active, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/classify, name=active-phase0, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

train: /content/data_active/train... found 6030 images in 22 classes ✅ 
val: /content/data_active/val... found 2511 images in 22 classes ✅ 
test: /content/data_active/test... found 2512 images in 22 classes ✅ 
Overriding model.yaml nc=80 with nc=22

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.

100%|██████████| 5.35M/5.35M [00:00<00:00, 95.9MB/s]


...

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/classify/active-phase4
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:07<00:00, 10.82it/s]


                   all      0.848      0.997

1 epochs completed in 0.026 hours.
Optimizer stripped from runs/classify/active-phase4/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/active-phase4/weights/best.pt, 3.2MB

Validating runs/classify/active-phase4/weights/best.pt...
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n-cls summary (fused): 47 layers, 1,554,206 parameters, 0 gradients, 3.2 GFLOPs
train: /content/data_active/train... found 14070 images in 22 classes ✅ 
val: /content/data_active/val... found 2511 images in 22 classes ✅ 
test: /content/data_active/test... found 2512 images in 22 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:07<00:00, 10.94it/s]


                   all      0.849      0.997
Speed: 0.5ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/active-phase4
Phase 5 training completed
